# Парсинг данных с сайта **avito.ru**

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
import requests
import random
from urllib.parse import quote
import os
import re
import pandas as pd


## Подготовка

Для того, чтобы сайт не мог так просто заподозрить, что мы берем данные с него, возьмем заголовки с реального браузера.

In [2]:
with open('user_agents.txt', 'r') as parse_items:
    user_agents = list(map(lambda x: x.strip(), parse_items.readlines()))

Указываем ссылку, с которой будем брать данные `data_url` и ссылку-якорь, которая позволит создавать ссылки на страницы с объявлениями `base_url`.

In [3]:
data_url = 'https://www.avito.ru/all/kvartiry/prodam-ASgBAgICAUSSA8YQ'
base_url = 'https://www.avito.ru'

# Очень желательно поставить реальную куку с сайта, иначе очень быстро забанят
# cookie = кука
cookie = кука

# не рекомендуется создавать данные больше 4000, так как реальных объявлений квартир не так много и со временем они начинают повторяться
dataset_size = 4000
links = set()

# точка означает, что будет записано в текущую директорию
folder_to_save_csvs = 'csvs'
folder_to_save_htmls = 'html_pages'

## Приступаем к делу (парсим ссылки на объявления)

Для начала нам придется достать все ссылки с объявлениями

In [4]:
os.path.exists(folder_to_save_htmls) or os.makedirs(folder_to_save_htmls)
os.path.exists(folder_to_save_csvs) or os.makedirs(folder_to_save_csvs)
print('Созданы папки для сохранения данных')

Созданы папки для сохранения данных


In [5]:
pbar = tqdm(total=dataset_size, desc='Links obtained')

current_page = 1
while len(links) < dataset_size:

    page = requests.get(
        data_url,
        # для получения следующей страницы нужно передать параметр p с номером страницы, которую мы будем получать
        params={'p': current_page},
        headers={
            'User-Agent': quote(random.choice(user_agents)),

            # Очень желательно поставить реальную куку с сайта, иначе очень быстро забанят
            'Cookie': cookie,
        }
    )

    soup = BeautifulSoup(page.text, 'html.parser')
    # print(soup.prettify())
    # break
    prev_len = len(links)

    for item in soup.find_all(name='a', attrs={'data-marker': 'item-title'}):
        if str(item.get('href')).startswith('http'):
            continue

        # Получаем ссылку на одно из объявлений
        cur_link = f'{base_url}{item.get("href")}'.strip()

        # Если эта ссылка уже была, пропускаем
        if cur_link in links:
            continue

        links.add(cur_link)

        if len(links) >= dataset_size:
            break

    pbar.update(len(links) - prev_len)
    current_page += 1

    # задержка между запросами, чтобы не забанили
    sleep(3 + abs(random.uniform(0, 1)))

del pbar


Links obtained: 100%|██████████| 4000/4000 [07:28<00:00,  8.91it/s]


In [6]:
links = list(links)

После того, как все ссылки получены, записываем на в файл, для того, чтобы нечаянно не потерять их в будущем

In [7]:
print("Примеры ссылок:")
print(*links[:5], sep='\n')
print()
print("Всего ссылок {}".format(len(links)))

with open(f'{folder_to_save_csvs}/links.csv', 'w') as links_csv:
    links_csv.write('\n'.join(links))

Примеры ссылок:
https://www.avito.ru/moskva/kvartiry/2-k._apartamenty_68m_1131et._2697437340
https://www.avito.ru/selyatino/kvartiry/3-k._kvartira_72m_33et._2751575016
https://www.avito.ru/novo-talitsy/kvartiry/4-k._kvartira_592m_23et._2899169847
https://www.avito.ru/velikiy_novgorod/kvartiry/3-k._kvartira_782m_59et._2470065859
https://www.avito.ru/perm/kvartiry/4-k._kvartira_835m_1212et._2727751263

Всего ссылок 4000


## Достаем объявления

После того, как мы получили все ссылки на объявления, приступим к парсингу самих объявлений

Небольшая фукнция, которая проверяет наличие необходимых тегов на странице

In [8]:
def parse_items(x: str):
    if x is None:
        return False
    return 'params-paramsList__item' in x \
        or 'style-item-params-list-item' in x
    # or 'style-item-address__string' in x \
    # or 'style-item-map-wrapper' in x and 'style-expanded' in x

Скачиваем страницы аналогично тому, как мы скачивали ссылки на объявления, только теперь проверяем на валидность и сохраняем файлы.

Необходимость в скачивании нужна по нескольким причинам. Во-первых, для того, чтобы изменить таблицу с данными, не нужно заново качать страницы, достаточно изменить код их обрабатывающий. Во-вторых данные на сайте могут устареть: ранее работавшая ссылка может измениться, могут измениться данные по ссылке (иногда ссылка может просто перестать работать).

In [9]:
pbar = tqdm(total=len(links), desc='Links obtained')

for index, link in enumerate(links):
    res = requests.get(
        url=link,
        headers={
            'User-Agent': quote(random.choice(user_agents)),

            # Очень желательно поставить реальную куку с сайта, иначе очень быстро забанят
            'Cookie': cookie,
        }
    )

    soup = BeautifulSoup(res.text)

    parsed_data = soup.find_all(
        name='li',
        class_=lambda x: parse_items(x)
    )

    if len(parsed_data) == 0:
        print(f'Что то пошло не так со ссылкой {index}\n{link}')
        with open(f'{folder_to_save_csvs}/trouble_links.txt', 'a') as trouble_links:
            trouble_links.write(link + '\n')
        
        continue

    filename = f"{folder_to_save_htmls}/{link.split('/')[-1]}.html"
    with open(filename, 'w') as html_file:
        html_file.write(res.text)

    with open(f'{folder_to_save_csvs}/data.csv', 'a') as output_csv:
        output_csv.write(f'{link},{filename}\n')

    pbar.update()
    sleep(3 + abs(random.uniform(0, 1)))


Links obtained:   7%|▋         | 263/4000 [17:03<4:01:10,  3.87s/it]

Что то пошло не так со ссылкой 262
https://www.avito.ru/usinsk/kvartiry/1-k._kvartira_332m_45et._2627166168


Links obtained:  15%|█▍        | 581/4000 [37:47<3:47:44,  4.00s/it]

Что то пошло не так со ссылкой 580
https://www.avito.ru/moskva/kvartiry/1-k._kvartira_28m_99et._2888043126


Links obtained:  21%|██        | 821/4000 [53:24<3:35:41,  4.07s/it]

...

Links obtained:  90%|████████▉ | 3583/4000 [3:53:19<27:16,  3.92s/it]  

Что то пошло не так со ссылкой 3582
https://www.avito.ru/evpatoriya/kvartiry/3-k._kvartira_66m_39et._3101477779


Links obtained:  92%|█████████▏| 3672/4000 [3:59:06<21:14,  3.88s/it]

Что то пошло не так со ссылкой 3671
https://www.avito.ru/sochi/kvartiry/kvartira-studiya_289m_14et._2575605749


Links obtained: 100%|██████████| 4000/4000 [4:20:31<00:00,  3.83s/it]

## Парсим страницы

Теперь, когда у нас есть все необходимое, можно начать работать с данными.

Напишем функцию, превращающую html страничку в словарь с данными

In [12]:
# Напишем функцию, превращающую html страничку в словарь с данными
def parse_html(filename: str):

    with open(filename) as html_file:
        soup = BeautifulSoup(html_file.read())

    parsed_data = soup.find_all(
        name='li',
        class_=lambda x: parse_items(x)
    )
    
    if not len(parsed_data):
        return None

    parsed_data[0] = re.sub('<style.*<\/style>', '', str(parsed_data[0]))

    price = float(soup.find(name='span', attrs={
                  'itemprop': 'price'}).attrs['content'])
    currency = soup.find(name='span', attrs={
                         'itemprop': 'priceCurrency'}).attrs['content']

    parsed_data = {x.contents[0].contents[0]: x.contents[1]
                   for x in parsed_data[1:]}
    parsed_data['Цена'] = price
    parsed_data['Валюта'] = currency

    return parsed_data


parse_html(
    f'{folder_to_save_htmls}/{links[0].split("/")[-1]}.html')

{'Общая площадь': '68\xa0м²',
 'Площадь кухни': '15\xa0м²',
 'Этаж': '11 из 31',
 'Балкон или лоджия': 'балкон',
 'Тип комнат': 'изолированные, смежные',
 'Высота потолков': '3.2\xa0м',
 'Санузел': 'совмещенный',
 'Окна': 'во двор, на солнечную сторону',
 'Ремонт': 'дизайнерский',
 'Тёплый пол': 'есть',
 'Мебель': 'кухня, хранение одежды, спальные места',
 'Техника': 'кондиционер, холодильник, стиральная машина, посудомоечная машина',
 'Способ продажи': 'свободная',
 'Вид сделки': 'возможна ипотека',
 'Тип дома': 'монолитно-кирпичный',
 'Год постройки': '2015',
 'Этажей в доме': '31',
 'Пассажирский лифт': '3',
 'Грузовой лифт': '1',
 'Двор': 'закрытая территория, детская площадка, спортивная площадка',
 'Парковка': 'подземная, открытая во дворе, за шлагбаумом во дворе',
 'Цена': 24500000.0,
 'Валюта': 'RUB'}

Получим подобные словари для всех наших данных

In [13]:
all_dicts = []
with open(f'{folder_to_save_csvs}/data.csv') as data_csv:
    for line in data_csv.readlines():
        link, filename = line.strip().split(',')

        cur_dict = parse_html(filename)
        if cur_dict is None:
            continue
        
        cur_dict['Ссылка'] = link

        all_dicts.append(cur_dict)

for dict_ in all_dicts[:5]:
    print(dict_)

{'Общая площадь': '68\xa0м²', 'Площадь кухни': '15\xa0м²', 'Этаж': '11 из 31', 'Балкон или лоджия': 'балкон', 'Тип комнат': 'изолированные, смежные', 'Высота потолков': '3.2\xa0м', 'Санузел': 'совмещенный', 'Окна': 'во двор, на солнечную сторону', 'Ремонт': 'дизайнерский', 'Тёплый пол': 'есть', 'Мебель': 'кухня, хранение одежды, спальные места', 'Техника': 'кондиционер, холодильник, стиральная машина, посудомоечная машина', 'Способ продажи': 'свободная', 'Вид сделки': 'возможна ипотека', 'Тип дома': 'монолитно-кирпичный', 'Год постройки': '2015', 'Этажей в доме': '31', 'Пассажирский лифт': '3', 'Грузовой лифт': '1', 'Двор': 'закрытая территория, детская площадка, спортивная площадка', 'Парковка': 'подземная, открытая во дворе, за шлагбаумом во дворе', 'Цена': 24500000.0, 'Валюта': 'RUB', 'Ссылка': 'https://www.avito.ru/moskva/kvartiry/2-k._apartamenty_68m_1131et._2697437340'}
{'Общая площадь': '72\xa0м²', 'Площадь кухни': '13.4\xa0м²', 'Жилая площадь': '43\xa0м²', 'Этаж': '3 из 3', 'Ба

## Сохраняем данные

In [14]:
df = pd.DataFrame(all_dicts)
df.head()


,Общая площадь,Площадь кухни,Этаж,Балкон или лоджия,Тип комнат,Высота потолков,Санузел,Окна,Ремонт,Тёплый пол,...,Ссылка,Жилая площадь,В доме,Отделка,Название новостройки,Официальный застройщик,Тип участия,Срок сдачи,"Корпус, строение",Запланирован снос
0,68 м²,15 м²,11 из 31,балкон,"изолированные, смежные",3.2 м,совмещенный,"во двор, на солнечную сторону",дизайнерский,есть,...,https://www.avito.ru/moskva/kvartiry/2-k._apar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72 м²,13.4 м²,3 из 3,лоджия,изолированные,NaN,раздельный,NaN,NaN,NaN,...,https://www.avito.ru/selyatino/kvartiry/3-k._k...,43 м²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59.2 м²,6 м²,2 из 3,NaN,"изолированные, смежные",NaN,раздельный,NaN,косметический,NaN,...,https://www.avito.ru/novo-talitsy/kvartiry/4-k...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,78.2 м²,29.1 м²,5 из 9,балкон,изолированные,NaN,раздельный,"во двор, на улицу",евро,NaN,...,https://www.avito.ru/velikiy_novgorod/kvartiry...,NaN,газ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,83.5 м²,NaN,12 из 12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.avito.ru/perm/kvartiry/4-k._kvarti...,NaN,NaN,предчистовая,[ЖК «Талисман»],ООО СЗ «СИТИ Проект»,ДДУ по ФЗ 214,Сдача в 3 кв. 2024,NaN,NaN


In [15]:
df.to_csv(f'{folder_to_save_csvs}/flats.csv', index=True)
